In [111]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from tqdm import tqdm
from geopy.geocoders import Nominatim
import pickle
from shapely import geometry
from tqdm import tqdm

## Как работать с h3_9:

In [179]:
geo = {
    'type': 'Polygon',
    'coordinates': [
        [
            [37.813318999983238, -122.4089866999972145],
            [37.7866302000007224, -122.3805436999997056],
            [37.7198061999978478, -122.3544736999993603],
            [37.7076131999975672, -122.5123436999983966],
            [37.7835871999971715, -122.5247187000021967],
            [37.8151571999998453, -122.4798767000009008],
        ]
    ]
}

hexes = h3.polyfill(geo, 9)
hexes

{'8928309420fffff',
 '89283082d67ffff',
 '89283082b67ffff',
 '89283082e37ffff',
 '89283082a77ffff',
 '89283095c7bffff',
 '8928308290bffff',
 '89283095e4fffff',
 '89283082817ffff',
 '89283082b63ffff',
 '89283095007ffff',
 '8928309436fffff',
 '89283080d8bffff',
 '8928309594bffff',
 '89283094253ffff',
 '89283082d6fffff',
 '89283095887ffff',
 '8928308201bffff',
 '89283095b87ffff',
 '89283082ab3ffff',
 '8928309507bffff',
 '89283095b8fffff',
 '892830875bbffff',
 '89283095a0fffff',
 '89283095e0bffff',
 '89283095c27ffff',
 '89283082323ffff',
 '8928308210bffff',
 '89283080ca7ffff',
 '892830953d3ffff',
 '89283095123ffff',
 '89283095e87ffff',
 '89283095a73ffff',
 '89283095cc7ffff',
 '89283094273ffff',
 '89283082533ffff',
 '89283087587ffff',
 '8928309510bffff',
 '89283095b0fffff',
 '8928308282bffff',
 '8928309596fffff',
 '89283095d07ffff',
 '89283082987ffff',
 '892830942abffff',
 '89283082917ffff',
 '892830828b7ffff',
 '892830953c3ffff',
 '89283087457ffff',
 '89283095bb7ffff',
 '89283095ed3ffff',


In [4]:
h = '891106a9657ffff'

In [5]:
h3.h3_to_geo(h)

(59.96648884814553, 30.31301194723923)

In [6]:
h3_test = h3.h3_to_geo_boundary(h)
h3_test

((59.96555900587286, 30.3102869648344),
 (59.96473800165777, 30.31297586051135),
 (59.96566779246675, 30.31570091656956),
 (59.967418654016214, 30.315737140541707),
 (59.96823968409392, 30.313048037565093),
 (59.96730982675726, 30.310322917923106))

In [185]:
h3.geo_to_h3(60.18600981831053, 29.576714118375616, 9)

'8911044a157ffff'

## Как вывести маркер на карту

In [60]:
h3_test_1 = ((30.311958245114507, 60.178148715221994), (30.314663468848707, 60.17733089470366), (30.317405157887038, 60.17826254489239), (30.317441687573208, 60.18001208215914), (30.314736254022115, 60.1808299286797), (30.311994500608964, 60.17989821192894), (30.311958245114507, 60.178148715221994))
map2 = folium.Map((h3_test_1[1][1],h3_test_1[1][0]), zoom_start=14)
for pt in h3_test_1:
    marker = folium.Marker([pt[1], pt[0]]) #latitude,longitude
    map2.add_child(marker)
map2

## Наши данные

In [8]:
json_tele2 = []
with open('spb_1.geojson', 'r') as j:
    contents = json.loads(j.read())
for i in contents['features'][0]['geometry']['coordinates'][0]:
    json_tele2.append(i)
    
json_tele2

[[28.7759027, 58.8295867],
 [28.7720449, 58.8300631],
 [28.7678554, 58.8313061],
 [28.7652418, 58.8302743],
 [28.7666863, 58.8279358],
 [28.7694376, 58.8267666],
 [28.7657924, 58.8248407],
 [28.7469465, 58.8282794],
 [28.7459147, 58.830274],
 [28.7424068, 58.8319934],
 [28.720414, 58.8380285],
 [28.6666617, 58.8523167],
 [28.6634979, 58.8510098],
 [28.6612969, 58.8519039],
 [28.6681746, 58.8560308],
 [28.6681744, 58.8594011],
 [28.671338, 58.8639406],
 [28.669412, 58.8661415],
 [28.6674861, 58.8664166],
 [28.6623964, 58.866279],
 [28.6592326, 58.8658662],
 [28.6564133, 58.8556179],
 [28.6447897, 58.8523163],
 [28.6406283, 58.8553426],
 [28.636364, 58.8550674],
 [28.6338188, 58.8603634],
 [28.6278349, 58.8616013],
 [28.6218509, 58.8626329],
 [28.6219884, 58.8645588],
 [28.6117401, 58.8665532],
 [28.5978808, 58.8682037],
 [28.5968492, 58.8672408],
 [28.5998756, 58.8649711],
 [28.5895589, 58.8603626],
 [28.58667, 58.8627011],
 [28.5881831, 58.8641455],
 [28.5869449, 58.8664152],
 [28.5797

In [9]:
json_tele2_df = pd.DataFrame(json_tele2, columns = ['lon', 'lat']) 
json_tele2_df

,lon,lat
0,28.775903,58.829587
1,28.772045,58.830063
2,28.767855,58.831306
3,28.765242,58.830274
4,28.766686,58.827936
...,...,...
4402,28.800148,58.827386
4403,28.797328,58.825942
4404,28.782540,58.829656
4405,28.779754,58.828142


In [10]:
spb_hex = pd.read_excel('spb_hex.xlsx', sheet_name='spb_hex', header=0, na_values=['NA'])
spb_hex

,h3_9,age_20,age_20_30,age_30_40,age_40_60,age_60,male,female,car_owners,carsharing_user,not_citizen_rf,public_trnsprt_user,healthy_food_inter,salary_1,salary_2,salary_3,salary_4,salary_5
0,8911063244fffff,0,0,8,3,1,7,5,4,5,2,0,0,1,6,2,2,1
1,891106074d7ffff,0,13,3,2,1,4,17,16,1,17,0,0,2,1,14,0,1
2,8911062a87bffff,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,0
3,89110615e0fffff,0,2,11,20,15,46,19,14,0,22,1,0,33,12,2,7,4
4,892d721010bffff,0,0,2,1,1,3,1,0,0,0,0,0,2,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32417,8911030ce4bffff,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0
32418,891106c2bc7ffff,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0
32419,891160c0693ffff,0,0,0,0,2,2,0,0,0,2,0,0,0,0,0,0,0
32420,8910a810153ffff,0,0,1,1,0,1,1,2,0,2,0,0,0,1,0,0,1


## Add coordinates

In [11]:
spb_hex['coordinates'] = spb_hex.apply(lambda x:  h3.h3_to_geo(x['h3_9']), axis=1)
spb_hex.head(3)

,h3_9,age_20,age_20_30,age_30_40,age_40_60,age_60,male,female,car_owners,carsharing_user,not_citizen_rf,public_trnsprt_user,healthy_food_inter,salary_1,salary_2,salary_3,salary_4,salary_5,coordinates
0,8911063244fffff,0,0,8,3,1,7,5,4,5,2,0,0,1,6,2,2,1,"(60.0585773849189, 30.45608462129097)"
1,891106074d7ffff,0,13,3,2,1,4,17,16,1,17,0,0,2,1,14,0,1,"(59.877288933703944, 30.316576555332126)"
2,8911062a87bffff,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,0,"(59.97806009684688, 30.970278125099377)"


## Add coordinates in geo format

In [12]:
spb_hex['coordinates_geo'] = spb_hex.apply(lambda x:  gpd.points_from_xy(spb_hex.coordinates[0], spb_hex.coordinates[1]), axis=1)
spb_hex.head(3)

,h3_9,age_20,age_20_30,age_30_40,age_40_60,age_60,male,female,car_owners,carsharing_user,not_citizen_rf,public_trnsprt_user,healthy_food_inter,salary_1,salary_2,salary_3,salary_4,salary_5,coordinates,coordinates_geo
0,8911063244fffff,0,0,8,3,1,7,5,4,5,2,0,0,1,6,2,2,1,"(60.0585773849189, 30.45608462129097)","[POINT (60.0585773849189 59.877288933703944), ..."
1,891106074d7ffff,0,13,3,2,1,4,17,16,1,17,0,0,2,1,14,0,1,"(59.877288933703944, 30.316576555332126)","[POINT (60.0585773849189 59.877288933703944), ..."
2,8911062a87bffff,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,0,"(59.97806009684688, 30.970278125099377)","[POINT (60.0585773849189 59.877288933703944), ..."


## Add lat/lon in geo format

In [13]:
spb_hex['lat'] = spb_hex['coordinates'].apply(lambda x: x[0])
spb_hex['lon'] = spb_hex['coordinates'].apply(lambda x: x[1])
spb_hex.head(3)

,h3_9,age_20,age_20_30,age_30_40,age_40_60,age_60,male,female,car_owners,carsharing_user,...,healthy_food_inter,salary_1,salary_2,salary_3,salary_4,salary_5,coordinates,coordinates_geo,lat,lon
0,8911063244fffff,0,0,8,3,1,7,5,4,5,...,0,1,6,2,2,1,"(60.0585773849189, 30.45608462129097)","[POINT (60.0585773849189 59.877288933703944), ...",60.058577,30.456085
1,891106074d7ffff,0,13,3,2,1,4,17,16,1,...,0,2,1,14,0,1,"(59.877288933703944, 30.316576555332126)","[POINT (60.0585773849189 59.877288933703944), ...",59.877289,30.316577
2,8911062a87bffff,0,0,1,0,0,1,0,1,1,...,0,0,0,1,0,0,"(59.97806009684688, 30.970278125099377)","[POINT (60.0585773849189 59.877288933703944), ...",59.978060,30.970278


## Выгрузка данных с OSM

In [13]:
def visualize_hexagons(hexagons, color="red", folium_map=None):

    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=20, tiles='cartodbpositron')
    else:
        m = folium_map
        
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m

In [14]:
# выгрузим границы Санкт-Петербурга из OSM
cities = ['Санкт-Петербург']
polygon_krd = ox.features_from_place(cities, {'boundary':'administrative'}).reset_index()
polygon_krd = polygon_krd[(polygon_krd['name'] == 'Санкт-Петербург')]
polygon_krd

,element_type,osmid,highway,geometry,boundary,ref,addr:country,addr:region,name,name:be,...,old_name:mk,oktmo:user,contact:instagram,contact:vk,contact:website,oktmo,short_name:be,short_name:ru,short_name:uk,admin_centre
1256,relation,337422,NaN,"MULTIPOLYGON (((30.06506 59.66830, 30.07331 59...",administrative,RU-SPE,RU,NaN,Санкт-Петербург,Санкт-Пецярбург,...,Ленинград,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
def visualize_polygons(geometry):
    
    lats, lons = get_lat_lon(geometry)
    
    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')
    
    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)
    
    return m

# выводим центроиды полигонов
def get_lat_lon(geometry):
        
    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon

In [17]:
def create_hexagons(geoJson):
    
    polyline = geoJson['coordinates'][0]

    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=9,color="green")
    m.add_child(my_PolyLine)

    hexagons = list(h3.polyfill(geoJson, 9))
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=3,color='red')
        m.add_child(my_PolyLine)
        
    polylines_x = []
    for j in range(len(polylines)):
        a = np.column_stack((np.array(polylines[j])[:,1],np.array(polylines[j])[:,0])).tolist()
        polylines_x.append([(a[i][0], a[i][1]) for i in range(len(a))])
        
    polygons_hex = pd.Series(polylines_x).apply(lambda x: Polygon(x))
        
    return m, polygons_hex, polylines
# polygon_hex , polylines - геометрии гексагонов в разных форматах

## Гексагоны

In [36]:
def hexagons_h3_9(geoJson):
    
    polyline = geoJson['coordinates'][0]

    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=9,color="green")
    m.add_child(my_PolyLine)

    hexagons = list(h3.polyfill(geoJson, 9))        
    return hexagons
# polygon_hex , polylines - геометрии гексагонов в разных форматах

In [56]:
geoJson_test = json.loads(gpd.GeoSeries(polygon_krd['geometry']).to_json())
geoJson_test

{'type': 'FeatureCollection',
 'features': [{'id': '1256',
   'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[30.0650571, 59.6683027],
       [30.073311, 59.6673744],
       [30.082713, 59.6760822],
       [30.0839517, 59.6802946],
       [30.0728262, 59.6797451],
       [30.065169, 59.677984],
       [30.065326, 59.6763773],
       [30.0650571, 59.6683027]]],
     [[[29.4388996, 60.1764284],
       [29.4395947, 60.1767224],
       [29.4410146, 60.1773395],
       [29.4427186, 60.17808],
       [29.4444021, 60.1787114],
       [29.4451149, 60.1789787],
       [29.4472437, 60.1795612],
       [29.4473759, 60.1795939],
       [29.4490735, 60.1799659],
       [29.4530873, 60.180701],
       [29.4541514, 60.180831],
       [29.4549966, 60.1809343],
       [29.4569952, 60.1810815],
       [29.4572812, 60.1810886],
       [29.4631727, 60.1812346],
       [29.4634771, 60.181349],
       [29.4667593, 60.1813821],
       [29.469156, 60.1815

In [59]:
m1 = folium.Map([43, -100], zoom_start=4)
folium.GeoJson(geoJson_test).add_to(m1)
m1

In [18]:
geoJson_test = json.loads(gpd.GeoSeries(polygon_krd['geometry']).to_json())
geoJson_test2 = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson_test['features'][0]['geometry']['coordinates'][1][0])[:, 1],
                                                      np.array(geoJson_test['features'][0]['geometry']['coordinates'][1][0])[:, 0])).tolist()]}
geoJson_test2

{'type': 'Polygon',
 'coordinates': [[[60.1764284, 29.4388996],
   [60.1767224, 29.4395947],
   [60.1773395, 29.4410146],
   [60.17808, 29.4427186],
   [60.1787114, 29.4444021],
   [60.1789787, 29.4451149],
   [60.1795612, 29.4472437],
   [60.1795939, 29.4473759],
   [60.1799659, 29.4490735],
   [60.180701, 29.4530873],
   [60.180831, 29.4541514],
   [60.1809343, 29.4549966],
   [60.1810815, 29.4569952],
   [60.1810886, 29.4572812],
   [60.1812346, 29.4631727],
   [60.181349, 29.4634771],
   [60.1813821, 29.4667593],
   [60.1815371, 29.469156],
   [60.1833957, 29.4683198],
   [60.183608, 29.4699632],
   [60.1852423, 29.4686534],
   [60.1837358, 29.4600232],
   [60.1848357, 29.4589751],
   [60.1849678, 29.456661],
   [60.1849348, 29.4544576],
   [60.1858651, 29.4511692],
   [60.1869936, 29.4509367],
   [60.187401, 29.4503942],
   [60.188045, 29.4498627],
   [60.1885901, 29.4495591],
   [60.1887432, 29.448637],
   [60.1889848, 29.447724],
   [60.1903878, 29.4271878],
   [60.1907379, 29.4

In [19]:
geoJson = json.loads(gpd.GeoSeries(polygon_krd['geometry']).to_json())
#print(geoJson)
geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['features'][0]['geometry']['coordinates'][1][0])[:, 1],
                                                      np.array(geoJson['features'][0]['geometry']['coordinates'][1][0])[:, 0])).tolist()]}
m, polygons, polylines = create_hexagons(geoJson)

In [37]:
geoJson = json.loads(gpd.GeoSeries(polygon_krd['geometry']).to_json())
#print(geoJson)
geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['features'][0]['geometry']['coordinates'][1][0])[:, 1],
                                                      np.array(geoJson['features'][0]['geometry']['coordinates'][1][0])[:, 0])).tolist()]}
hexagons = hexagons_h3_9(geoJson)

In [41]:
df_hexagons = pd.DataFrame(hexagons)
df_hexagons.to_excel('hexagons.xlsx', index = False)

C:\Econometrics\Temp\ipykernel_24736\2535085159.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_hexagons.to_excel('hexagons.xlsx', index = False)


## Сохраняем данные для последюущего быстрого использования

In [ ]:
# для dataframe и object
polygons.to_csv('polygons_OSM.xlsx', index = False)

In [ ]:
# для list
file_name = "polylines_OSM.pkl"

open_file = open(file_name, "wb")
pickle.dump(polylines, open_file)
open_file.close()

In [ ]:
# Чтобы октрыть файл
open_file = open("polylines_tele2.pkl", "rb")
loaded_list = pickle.load(open_file)
open_file.close()

## Непосредственно выгрузка из OSM

### Границы СПБ из ОСМ

In [18]:
def osm_query(tag, city):
    gdf = ox.features_from_place(city, tag).reset_index()
    gdf['city'] = np.full(len(gdf), city.split(',')[0])
    gdf['object'] = np.full(len(gdf), list(tag.keys())[0])
    gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])
    gdf = gdf[['city', 'object', 'type', 'geometry']]
    return gdf
  
 # Выгрузим интересующие нас категории объектов 
tags = [
        {'building' : 'apartments'}, {'building' : 'detached'}, 
        {'building' : 'dormitory'}, {'building' : 'hotel'}, 
        {'building' : 'house'}, {'building' : 'semidetached_house'}, 
        {'building' : 'terrace'},  {'building' : 'commercial'},
        {'building' : 'office'},  {'building' : 'terrace'},  
        {'building' : 'terrace'}, {'building':'retail'}, 
        {'building':'train_station'},
        {'highway' : 'bus_stop'}, {'footway':'crossing'},
        {'amenity':'cafe'}, {'amenity':'fast_food'}, 
        {'amenity':'restaurant'}, {'amenity':'college'}, 
        {'amenity':'language_school'},  {'amenity':'school'},  
        {'amenity':'university'},  {'amenity':'atm'},  
        {'amenity':'bank'},  {'amenity':'clinic'},  
        {'amenity':'hospital'},  {'amenity':'pharmacy'},  
        {'amenity':'theatre'},  {'amenity':'townhall'},  
        {'amenity':'bench'}, 
       ]
cities = ['Санкт-Петербург, Россия']

gdfs = []
for city in cities:
    for tag in tags:
        gdfs.append(osm_query(tag, city))
        
# посмотрим что получилось
data_poi = pd.concat(gdfs)
data_poi.groupby(['city','object','type'], as_index = False).agg({'geometry':'count'})

# добавим координаты/центроиды
lat, lon = get_lat_lon(data_poi['geometry'])
data_poi['lat'] = lat
data_poi['lon'] = lon
data_poi

C:\Econometrics\Temp\ipykernel_15524\233577974.py:15: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
C:\Econometrics\Temp\ipykernel_15524\233577974.py:16: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


,city,object,type,geometry,lat,lon
0,Санкт-Петербург,building,apartments,POINT (30.40388 60.00498),60.004981,30.403879
1,Санкт-Петербург,building,apartments,"POLYGON ((30.40320 60.04657, 30.40350 60.04651...",60.046459,30.403285
2,Санкт-Петербург,building,apartments,"POLYGON ((30.40161 60.04480, 30.40154 60.04471...",60.044930,30.400725
3,Санкт-Петербург,building,apartments,"POLYGON ((30.39701 60.04574, 30.39963 60.04518...",60.045510,30.398366
4,Санкт-Петербург,building,apartments,"POLYGON ((30.39747 60.04607, 30.39741 60.04599...",60.046401,30.397626
...,...,...,...,...,...,...
14609,Санкт-Петербург,amenity,bench,"LINESTRING (30.39300 59.94652, 30.39297 59.946...",59.946507,30.392972
14610,Санкт-Петербург,amenity,bench,"LINESTRING (30.30618 59.96646, 30.30619 59.96652)",59.966488,30.306183
14611,Санкт-Петербург,amenity,bench,"LINESTRING (30.30637 59.96645, 30.30639 59.96651)",59.966476,30.306380
14612,Санкт-Петербург,amenity,bench,"LINESTRING (30.47962 59.92509, 30.47964 59.925...",59.925074,30.479709


In [19]:
data_poi['h3_9_osm'] = data_poi.apply(lambda x:  h3.geo_to_h3(x.lat, x.lon, 9), axis=1)
data_poi

,city,object,type,geometry,lat,lon,h3_9_osm
0,Санкт-Петербург,building,apartments,POINT (30.40388 60.00498),60.004981,30.403879,891106a9e6bffff
1,Санкт-Петербург,building,apartments,"POLYGON ((30.40320 60.04657, 30.40350 60.04651...",60.046459,30.403285,891106a9d77ffff
2,Санкт-Петербург,building,apartments,"POLYGON ((30.40161 60.04480, 30.40154 60.04471...",60.044930,30.400725,891106a9d73ffff
3,Санкт-Петербург,building,apartments,"POLYGON ((30.39701 60.04574, 30.39963 60.04518...",60.045510,30.398366,891106a9d73ffff
4,Санкт-Петербург,building,apartments,"POLYGON ((30.39747 60.04607, 30.39741 60.04599...",60.046401,30.397626,891106a9d0fffff
...,...,...,...,...,...,...,...
14609,Санкт-Петербург,amenity,bench,"LINESTRING (30.39300 59.94652, 30.39297 59.946...",59.946507,30.392972,89110606edbffff
14610,Санкт-Петербург,amenity,bench,"LINESTRING (30.30618 59.96646, 30.30619 59.96652)",59.966488,30.306183,891106a96cfffff
14611,Санкт-Петербург,amenity,bench,"LINESTRING (30.30637 59.96645, 30.30639 59.96651)",59.966476,30.306380,891106a96cfffff
14612,Санкт-Петербург,amenity,bench,"LINESTRING (30.47962 59.92509, 30.47964 59.925...",59.925074,30.479709,891106043bbffff


In [23]:
data_poi.to_excel("ОСМ_объекты.xlsx", index = False)

C:\Econometrics\Temp\ipykernel_24736\2683694223.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  data_poi.to_excel("ОСМ_объекты.xlsx", index = False)


### Достаем районы СПб

In [49]:
# Function to clean the GeoDataFrame to ensure it can be saved without errors
def clean_gdf_for_saving(gdf):
    for col in gdf.columns:
        # Convert lists to strings
        if gdf[col].apply(lambda x: isinstance(x, list)).any():
            gdf[col] = gdf[col].astype(str)
    return gdf

# Function to retrieve the administrative boundaries of a city
def get_districts(city_name):
    # Use OSMnx to download the data for administrative boundaries
    # Note: As of my last training data in April 2023, OSMnx's `geometries_from_place` function may be deprecated.
    # You may need to use the `geometries.geometries_from_place` function instead if that is the case.
    gdf = ox.features_from_place(city_name, tags={'boundary': 'administrative', 'admin_level': '9'})
    
    # Clean the GeoDataFrame to ensure compatibility with GeoJSON format
    gdf_cleaned = clean_gdf_for_saving(gdf)
    
    return gdf_cleaned

In [ ]:
stp_districts_gdf = get_districts("Saint Petersburg, Russia")
stp_districts_gdf.to_csv('stp_districts_gdf_spb.csv', index = False)
df_base = pd.read_csv('stp_districts_gdf_spb.csv')

In [62]:
districts_polygons_df = df_base[df_base['geometry'].str.startswith(('POLYGON', 'MULTIPOLYGON'))]
SPB_and_lenoblast_districts = districts_polygons_df[districts_polygons_df['name'].str.contains('район', na=False)][['geometry', 'addr:region', 'name']]
SPB_districts = SPB_and_lenoblast_districts[SPB_and_lenoblast_districts['addr:region'].str.contains('Санкт-Петербург', na=False)][['geometry', 'addr:region', 'name']]
SPB_districts.head()

,geometry,addr:region,name
1156,"POLYGON ((30.4411207 59.8293735, 30.4433336 59...",Санкт-Петербург,Колпинский район
1162,"POLYGON ((30.2182227 59.6730689, 30.219183 59....",Санкт-Петербург,Пушкинский район
1163,"POLYGON ((30.1987721 59.8016053, 30.1990074 59...",Санкт-Петербург,Московский район
1172,"MULTIPOLYGON (((30.0650571 59.6683027, 30.0733...",Санкт-Петербург,Красносельский район
1178,"POLYGON ((30.3633101 59.9138661, 30.3695106 59...",Санкт-Петербург,Невский район


In [66]:
# replace string geometry representations with shapely geometries
SPB_districts['geometry'] = gpd.GeoSeries.from_wkt(SPB_districts['geometry'])

# initialize GeoDataFrame with the result
# ('geometry' is the default geometry column name)
SPB_districts = gpd.GeoDataFrame(SPB_districts)

In [84]:
SPB_districts = SPB_districts[['geometry', 'name']]
SPB_districts

,geometry,name
1156,"POLYGON ((30.44112 59.82937, 30.44333 59.82980...",Колпинский район
1162,"POLYGON ((30.21822 59.67307, 30.21918 59.67361...",Пушкинский район
1163,"POLYGON ((30.19877 59.80161, 30.19901 59.80180...",Московский район
1172,"MULTIPOLYGON (((30.06506 59.66830, 30.07331 59...",Красносельский район
1178,"POLYGON ((30.36331 59.91387, 30.36951 59.91412...",Невский район
1185,"POLYGON ((30.33258 59.91305, 30.33352 59.91337...",Фрунзенский район
1190,"POLYGON ((30.25023 59.90129, 30.25028 59.90132...",Адмиралтейский район
1191,"POLYGON ((30.18468 59.95472, 30.18473 59.95481...",Василеостровский район
1192,"POLYGON ((30.16799 59.88702, 30.16817 59.88725...",Кировский район
1193,"POLYGON ((30.39591 59.92574, 30.39428 59.92902...",Красногвардейский район


### Наземный трансорт

In [24]:
transport = pd.read_excel("Наземный транспорт.xlsx", sheet_name="Sheet1")
transport.head()

,№,Вид транспортного средства,Тип объекта,Наименование остановки,Официальное наименование,Расположение,Маршруты,lat,lon
0,1,Автобус,Остановка,10 КМ,"10 КМ [145, 165, 482, 482а, 482в, 484, 484б]",Таллинское ш,"145, 145Б, 147, 165, 245, 442, 458, 458Б, 481,...",59.773372,30.125111
1,2,Автобус,Остановка,12-13 ЛИНИИ В.О,12-13 ЛИНИИ В.О,"Средний пр., 12-13-я линии","249, 6",59.941121,30.271757
2,3,Автобус,Остановка,16 И 17 ЛИНИИ,"16 И 17 ЛИНИИ [7, 1]","Большой пр. В.О., Большой ВО Проспект","1, 128, 151, 262, 42, 7",59.935554,30.271147
3,4,Автобус,Остановка,МУЗЕЙ ГОРОДСКОГО НАЗЕМНОГО ТРАНСПОРТА,МУЗЕЙ ГОРОДСКОГО НАЗЕМНОГО ТРАНСПОРТА,"Средний пр., 18-19-я линии","249, 6",59.938798,30.264141
4,5,Автобус,Остановка,"1-Я И КАДЕТСКАЯ ЛИНИЯ,УГ. СРЕДНЕГО ПР","1-Я И КАДЕТСКАЯ ЛИНИЯ,УГ. СРЕДНЕГО ПР. [6]","1-я и Кадетская линии, Репина ул","24, 47, 6",59.944538,30.287008


In [25]:
transport_gdf = gpd.GeoDataFrame(transport, geometry=gpd.points_from_xy(transport.lon, transport.lat))
transport_gdf.head()

,№,Вид транспортного средства,Тип объекта,Наименование остановки,Официальное наименование,Расположение,Маршруты,lat,lon,geometry
0,1,Автобус,Остановка,10 КМ,"10 КМ [145, 165, 482, 482а, 482в, 484, 484б]",Таллинское ш,"145, 145Б, 147, 165, 245, 442, 458, 458Б, 481,...",59.773372,30.125111,POINT (30.12511 59.77337)
1,2,Автобус,Остановка,12-13 ЛИНИИ В.О,12-13 ЛИНИИ В.О,"Средний пр., 12-13-я линии","249, 6",59.941121,30.271757,POINT (30.27176 59.94112)
2,3,Автобус,Остановка,16 И 17 ЛИНИИ,"16 И 17 ЛИНИИ [7, 1]","Большой пр. В.О., Большой ВО Проспект","1, 128, 151, 262, 42, 7",59.935554,30.271147,POINT (30.27115 59.93555)
3,4,Автобус,Остановка,МУЗЕЙ ГОРОДСКОГО НАЗЕМНОГО ТРАНСПОРТА,МУЗЕЙ ГОРОДСКОГО НАЗЕМНОГО ТРАНСПОРТА,"Средний пр., 18-19-я линии","249, 6",59.938798,30.264141,POINT (30.26414 59.93880)
4,5,Автобус,Остановка,"1-Я И КАДЕТСКАЯ ЛИНИЯ,УГ. СРЕДНЕГО ПР","1-Я И КАДЕТСКАЯ ЛИНИЯ,УГ. СРЕДНЕГО ПР. [6]","1-я и Кадетская линии, Репина ул","24, 47, 6",59.944538,30.287008,POINT (30.28701 59.94454)


In [26]:
transport_gdf = transport_gdf[["Вид транспортного средства", "Наименование остановки", "Расположение", "lat", "lon", "geometry"]]
transport_gdf

,Вид транспортного средства,Наименование остановки,Расположение,lat,lon,geometry
0,Автобус,10 КМ,Таллинское ш,59.773372,30.125111,POINT (30.12511 59.77337)
1,Автобус,12-13 ЛИНИИ В.О,"Средний пр., 12-13-я линии",59.941121,30.271757,POINT (30.27176 59.94112)
2,Автобус,16 И 17 ЛИНИИ,"Большой пр. В.О., Большой ВО Проспект",59.935554,30.271147,POINT (30.27115 59.93555)
3,Автобус,МУЗЕЙ ГОРОДСКОГО НАЗЕМНОГО ТРАНСПОРТА,"Средний пр., 18-19-я линии",59.938798,30.264141,POINT (30.26414 59.93880)
4,Автобус,"1-Я И КАДЕТСКАЯ ЛИНИЯ,УГ. СРЕДНЕГО ПР","1-я и Кадетская линии, Репина ул",59.944538,30.287008,POINT (30.28701 59.94454)
...,...,...,...,...,...,...
8356,Автобус,УЛ. СТАНЮКОВИЧА,"улица Станюковича, улица Гидростроителей",60.010040,29.721394,POINT (29.72139 60.01004)
8357,Автобус,УЛ. ГИДРОСТРОИТЕЛЕЙ,"улица Гидростроителей, улица Станюковича",60.010139,29.716904,POINT (29.71690 60.01014)
8358,Автобус,"КРОНШТАДТСКОЕ ШОССЕ, 38","Кронштадтское шоссе, улица Литке",60.011040,29.726387,POINT (29.72639 60.01104)
8359,Троллейбус,"Северный пр., угол ул. Софьи Ковалевской, по т...","Северный проспект, улица Софьи Ковалевской",60.019513,30.416523,POINT (30.41652 60.01951)


In [27]:
transport_gdf['h3_9_transport'] = transport_gdf.apply(lambda x:  h3.geo_to_h3(x.lat, x.lon, 9), axis=1)
transport_gdf

,Вид транспортного средства,Наименование остановки,Расположение,lat,lon,geometry,h3_9_transport
0,Автобус,10 КМ,Таллинское ш,59.773372,30.125111,POINT (30.12511 59.77337),89110611d4bffff
1,Автобус,12-13 ЛИНИИ В.О,"Средний пр., 12-13-я линии",59.941121,30.271757,POINT (30.27176 59.94112),89110614aa3ffff
2,Автобус,16 И 17 ЛИНИИ,"Большой пр. В.О., Большой ВО Проспект",59.935554,30.271147,POINT (30.27115 59.93555),89110614a17ffff
3,Автобус,МУЗЕЙ ГОРОДСКОГО НАЗЕМНОГО ТРАНСПОРТА,"Средний пр., 18-19-я линии",59.938798,30.264141,POINT (30.26414 59.93880),89110614a87ffff
4,Автобус,"1-Я И КАДЕТСКАЯ ЛИНИЯ,УГ. СРЕДНЕГО ПР","1-я и Кадетская линии, Репина ул",59.944538,30.287008,POINT (30.28701 59.94454),89110614bcbffff
...,...,...,...,...,...,...,...
8356,Автобус,УЛ. СТАНЮКОВИЧА,"улица Станюковича, улица Гидростроителей",60.010040,29.721394,POINT (29.72139 60.01004),891106871c7ffff
8357,Автобус,УЛ. ГИДРОСТРОИТЕЛЕЙ,"улица Гидростроителей, улица Станюковича",60.010139,29.716904,POINT (29.71690 60.01014),891106871c7ffff
8358,Автобус,"КРОНШТАДТСКОЕ ШОССЕ, 38","Кронштадтское шоссе, улица Литке",60.011040,29.726387,POINT (29.72639 60.01104),8911068711bffff
8359,Троллейбус,"Северный пр., угол ул. Софьи Ковалевской, по т...","Северный проспект, улица Софьи Ковалевской",60.019513,30.416523,POINT (30.41652 60.01951),891106a984fffff


In [28]:
transport_gdf.to_csv('Транспорт.csv', index = False, encoding="utf-8-sig")

### Спортивные клубы

In [29]:
sport_clubs = pd.read_excel("fintess_clubs_spb.xlsx", sheet_name="Sheet1")
sport_clubs.head()

,Наименование,Описание,Рубрики,Адрес,Комментарий к адресу,Почтовый индекс,Микрорайон,Район,Город,Округ,...,WhatsApp 3,Viber,Telegram 1,Telegram 2,Telegram 3,YouTube,lat,lon,2GIS URL,Тип
0,#Sekta,офис,Фитнес-клубы; Велнес-залы,"Ефимова, 4а",3 этаж,190031.0,Сенной,Адмиралтейский район,Санкт-Петербург,NaN,...,NaN,NaN,https://t.me/sektaschool_bot,NaN,NaN,https://youtube.com/user/sektaschool,59.925725,30.321110,https://2gis.com/firm/70000001065994007,branch
1,#Sekta,офис,Фитнес-клубы; Велнес-залы,"13-я линия В.О., 80",цокольный этаж,199178.0,Васильевский,Василеостровский район,Санкт-Петербург,NaN,...,NaN,NaN,https://t.me/sektaschool_bot,NaN,NaN,https://youtube.com/user/sektaschool,59.947709,30.262752,https://2gis.com/firm/70000001065918917,branch
2,#ТрансФормАция,фитнес-студия,Фитнес-клубы,"проспект Ленина, 3",1 этаж,187000.0,NaN,NaN,Тосно,Тосненский муниципальный район,...,NaN,NaN,NaN,NaN,NaN,NaN,59.549946,30.858361,https://2gis.com/firm/70000001037573216,branch
3,101,спортклуб,Фитнес-клубы; Тренажёрные залы,"Социалистическая, 101",цокольный этаж; вход с торца,188643.0,NaN,NaN,Всеволожск,Всеволожский муниципальный район,...,NaN,NaN,NaN,NaN,NaN,NaN,60.018238,30.647618,https://2gis.com/firm/70000001055952550,branch
4,14-79 Jumping,студия,Фитнес-клубы,"Новгородский проспект, 8",NaN,196624.0,Шушары пос.,Пушкинский район,Санкт-Петербург,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.809280,30.368297,https://2gis.com/firm/70000001067709947,branch


In [30]:
sport_clubs_gdf = gpd.GeoDataFrame(sport_clubs, geometry=gpd.points_from_xy(sport_clubs.lon, sport_clubs.lat))
sport_clubs_gdf.head()

,Наименование,Описание,Рубрики,Адрес,Комментарий к адресу,Почтовый индекс,Микрорайон,Район,Город,Округ,...,Viber,Telegram 1,Telegram 2,Telegram 3,YouTube,lat,lon,2GIS URL,Тип,geometry
0,#Sekta,офис,Фитнес-клубы; Велнес-залы,"Ефимова, 4а",3 этаж,190031.0,Сенной,Адмиралтейский район,Санкт-Петербург,NaN,...,NaN,https://t.me/sektaschool_bot,NaN,NaN,https://youtube.com/user/sektaschool,59.925725,30.321110,https://2gis.com/firm/70000001065994007,branch,POINT (30.32111 59.92572)
1,#Sekta,офис,Фитнес-клубы; Велнес-залы,"13-я линия В.О., 80",цокольный этаж,199178.0,Васильевский,Василеостровский район,Санкт-Петербург,NaN,...,NaN,https://t.me/sektaschool_bot,NaN,NaN,https://youtube.com/user/sektaschool,59.947709,30.262752,https://2gis.com/firm/70000001065918917,branch,POINT (30.26275 59.94771)
2,#ТрансФормАция,фитнес-студия,Фитнес-клубы,"проспект Ленина, 3",1 этаж,187000.0,NaN,NaN,Тосно,Тосненский муниципальный район,...,NaN,NaN,NaN,NaN,NaN,59.549946,30.858361,https://2gis.com/firm/70000001037573216,branch,POINT (30.85836 59.54995)
3,101,спортклуб,Фитнес-клубы; Тренажёрные залы,"Социалистическая, 101",цокольный этаж; вход с торца,188643.0,NaN,NaN,Всеволожск,Всеволожский муниципальный район,...,NaN,NaN,NaN,NaN,NaN,60.018238,30.647618,https://2gis.com/firm/70000001055952550,branch,POINT (30.64762 60.01824)
4,14-79 Jumping,студия,Фитнес-клубы,"Новгородский проспект, 8",NaN,196624.0,Шушары пос.,Пушкинский район,Санкт-Петербург,NaN,...,NaN,NaN,NaN,NaN,NaN,59.809280,30.368297,https://2gis.com/firm/70000001067709947,branch,POINT (30.36830 59.80928)


In [31]:
sport_clubs_gdf = sport_clubs_gdf[["Наименование", "Описание", "Адрес", "lat", "lon", "geometry"]]
sport_clubs_gdf

,Наименование,Описание,Адрес,lat,lon,geometry
0,#Sekta,офис,"Ефимова, 4а",59.925725,30.321110,POINT (30.32111 59.92572)
1,#Sekta,офис,"13-я линия В.О., 80",59.947709,30.262752,POINT (30.26275 59.94771)
2,#ТрансФормАция,фитнес-студия,"проспект Ленина, 3",59.549946,30.858361,POINT (30.85836 59.54995)
3,101,спортклуб,"Социалистическая, 101",60.018238,30.647618,POINT (30.64762 60.01824)
4,14-79 Jumping,студия,"Новгородский проспект, 8",59.809280,30.368297,POINT (30.36830 59.80928)
...,...,...,...,...,...,...
846,Фитнес Двор,фитнес-центр,"проспект Обуховской Обороны, 116",59.864425,30.472444,POINT (30.47244 59.86442)
847,Фитнес и точка,NaN,"улица Рощинская, 13а",59.577821,30.132593,POINT (30.13259 59.57782)
848,Фитнес Леди,NaN,"Ленинский проспект, 90",59.856369,30.207502,POINT (30.20750 59.85637)
849,Фитнес Лига,спортивный клуб,"Бумажная, 9",59.905496,30.270479,POINT (30.27048 59.90550)


In [32]:
sport_clubs_gdf['h3_9_club'] = sport_clubs_gdf.apply(lambda x:  h3.geo_to_h3(x.lat, x.lon, 9), axis=1)
sport_clubs_gdf

,Наименование,Описание,Адрес,lat,lon,geometry,h3_9_club
0,#Sekta,офис,"Ефимова, 4а",59.925725,30.321110,POINT (30.32111 59.92572),891106066b3ffff
1,#Sekta,офис,"13-я линия В.О., 80",59.947709,30.262752,POINT (30.26275 59.94771),89110614847ffff
2,#ТрансФормАция,фитнес-студия,"проспект Ленина, 3",59.549946,30.858361,POINT (30.85836 59.54995),8911067814fffff
3,101,спортклуб,"Социалистическая, 101",60.018238,30.647618,POINT (30.64762 60.01824),89110631e8fffff
4,14-79 Jumping,студия,"Новгородский проспект, 8",59.809280,30.368297,POINT (30.36830 59.80928),89110600e07ffff
...,...,...,...,...,...,...,...
846,Фитнес Двор,фитнес-центр,"проспект Обуховской Обороны, 116",59.864425,30.472444,POINT (30.47244 59.86442),89110605077ffff
847,Фитнес и точка,NaN,"улица Рощинская, 13а",59.577821,30.132593,POINT (30.13259 59.57782),891106cc9cfffff
848,Фитнес Леди,NaN,"Ленинский проспект, 90",59.856369,30.207502,POINT (30.20750 59.85637),89110615073ffff
849,Фитнес Лига,спортивный клуб,"Бумажная, 9",59.905496,30.270479,POINT (30.27048 59.90550),891106158a3ffff


In [33]:
sport_clubs_gdf.to_csv('Спортивные_клубы.csv', index = False, encoding="utf-8-sig")

### Новые данные

In [5]:
yandex_objects = pd.read_excel("yandex_maps_final.xlsx", sheet_name="Sheet1")
parking_objects = pd.read_excel("Парковки_спб.xlsx", sheet_name="Sheet1")
metro_objects = pd.read_excel("Вестюбюли метро.xlsx", sheet_name="Sheet1")
cian_objects = pd.read_excel("Циан_аренда_с_координатами.xlsx", sheet_name="Sheet1")

In [6]:
yandex_objects['h3_9_yandex'] = yandex_objects.apply(lambda x:  h3.geo_to_h3(x.lat, x.lon, 9), axis=1)
yandex_objects

,id,coordinate,lon,lat,title,infos,category,Подкатегория,Категория,Для дупликатов,h3_9_yandex
0,26689477,"30.16816, 59.8155",30.168160,59.815500,Емг Северо-Запад,"Рабочая ул., 7, Красносельский район, историче...",742,Шиномонтаж,Авто,"Емг Северо-ЗападРабочая ул., 7, Красносельский...",89110610b7bffff
1,23888554,"30.167744, 59.815758",30.167744,59.815758,Мосавтошина,"Рабочая ул., 7Л, Красносельский район, историч...",742,Шиномонтаж,Авто,"МосавтошинаРабочая ул., 7Л, Красносельский рай...",89110610b7bffff
2,23856942,"30.142624, 59.825364",30.142624,59.825364,Икар,"просп. Народного Ополчения, 86<br>+7 (812) 643...",742,Шиномонтаж,Авто,"Икарпросп. Народного Ополчения, 86<br>+7 (812)...",89110610b97ffff
3,23802964,"30.152274, 59.822647",30.152274,59.822647,ProAutoSPB,"12, территория Станционный Посёлок, Санкт-Пете...",742,Шиномонтаж,Авто,"ProAutoSPB12, территория Станционный Посёлок, ...",89110610b13ffff
4,25525679,"30.173387, 59.824735",30.173387,59.824735,PitStop,"ул. Партизана Германа, 47, Санкт-Петербург<br>...",742,Шиномонтаж,Авто,"PitStopул. Партизана Германа, 47, Санкт-Петерб...",89110610b2bffff
...,...,...,...,...,...,...,...,...,...,...,...
594285,23491840,"30.489539, 60.110854",30.489539,60.110854,Суперхимчистка,"Ленинградская обл., Всеволожский р-н, Кузьмоло...",4986,Химчистки,Услуги,"СуперхимчисткаЛенинградская обл., Всеволожский...",891106ad2afffff
594286,23527874,"30.489719, 60.111361",30.489719,60.111361,"химчистка, прачечная","Кузьмоловский, улица Победы, 8",4986,Химчистки,Услуги,"химчистка, прачечнаяКузьмоловский, улица Побед...",891106ad2afffff
594287,23817880,"30.455457, 60.119659",30.455457,60.119659,Генералочка,"ул. Охтинская Перспектива, 13, д. Энколово, КП...",4986,Химчистки,Услуги,"Генералочкаул. Охтинская Перспектива, 13, д. Э...",891106ad0d7ffff
594288,23531884,"30.507748, 60.119813",30.507748,60.119813,Шиномонтаж,"Ленинградское ш., 50, д. Кузьмолово<br>ежеднев...",4986,Химчистки,Услуги,"ШиномонтажЛенинградское ш., 50, д. Кузьмолово<...",891106ad30bffff


In [7]:
parking_objects['h3_9_parking'] = parking_objects.apply(lambda x:  h3.geo_to_h3(x.lat, x.lon, 9), axis=1)
parking_objects

,№ автостоянки,Адрес,Тип автостоянки,Количество машиномест,"Площадь участка, м2",Район города,Ближайшая станция метро,Условия использования,Координаты,lat,lon,h3_9_parking
0,1,"Санкт-Петербург, проспект Энгельса, у д. 162, ...",Перехватывающая,155,4804.00,Выборгский,Парнас,Круглосуточно. Бесплатно при въезде с 06:01 до...,"60.067403,30.334694",60.067403,30.334694,891106a8107ffff
1,2,"Санкт-Петербург, проспект Просвещения, у д. 30...",Перехватывающая,43,1319.00,Выборгский,Проспект Просвещения,Круглосуточно. Бесплатно при въезде с 06:01 до...,"60.054322,30.32702",60.054322,30.327020,891106a8027ffff
2,3,"Санкт-Петербург, Гражданский проспект, у д. 11...",Перехватывающая,46,1423.00,Калининский,Гражданский проспект,Круглосуточно. Бесплатно при въезде с 06:01 до...,"60.035646,30.41789",60.035646,30.417890,891106a9837ffff
3,4,"Санкт-Петербург, Политехническая улица, у д. 2...",Перехватывающая,61,1413.00,Калининский,Политехническая,Круглосуточно. Бесплатно при въезде с 06:01 до...,"60.008229,30.370082",60.008229,30.370082,891106a916fffff
4,5,"Санкт-Петербург, проспект Стачек, у д. 47, лит...",Перехватывающая,50,2722.00,Кировский,Кировский завод,Круглосуточно. Бесплатно при въезде с 06:01 до...,"59.88067,30.261241",59.880670,30.261241,89110615e3bffff
...,...,...,...,...,...,...,...,...,...,...,...,...
57,58,"Санкт-Петербург, улица Пестеля (от наб.р. Фонт...",Зона платной парковки,67,887.75,Центральный,Чернышевская / Площадь Восстания / Маяковская,"Зона платной парковки № 1101. Пн-Вс, 08:00 - 2...","59.942694,30.344788",59.942694,30.344788,891106060b3ffff
58,59,"Санкт-Петербург, улица Радищева (от ул. Жуковс...",Зона платной парковки,112,1484.00,Центральный,Чернышевская / Площадь Восстания / Маяковская,"Зона платной парковки № 1126. Пн-Вс, 08:00 - 2...","59.938335,30.36327",59.938335,30.363270,8911060602bffff
59,60,"Санкт-Петербург, улица Рылеева (от Преображенс...",Зона платной парковки,106,1404.50,Центральный,Чернышевская / Площадь Восстания / Маяковская,"Зона платной парковки № 1105. Пн-Вс, 08:00 - 2...","59.9425,30.35635",59.942500,30.356350,89110606033ffff
60,61,"Санкт-Петербург, улица Чехова (от ул. Жуковско...",Зона платной парковки,70,927.50,Центральный,Чернышевская / Площадь Восстания / Маяковская,"Зона платной парковки № 1123. Пн-Вс, 08:00 - 2...","59.938,30.352872",59.938000,30.352872,89110606007ffff


In [8]:
metro_objects['h3_9_metro'] = metro_objects.apply(lambda x:  h3.geo_to_h3(x.lat, x.lon, 9), axis=1)
metro_objects

,№,Наименование,Адрес,Координаты,lat,lon,h3_9_metro
0,1,Проспект Ветеранов (вестибюль 1),"г.Санкт-Петербург, бульвар Новаторов, соор.2, ...","59.853406, 30.269578",59.853406,30.269578,89110602d53ffff
1,2,Проспект Ветеранов (вестибюль 2),"г.Санкт-Петербург, бульвар Новаторов, соор.2, ...","59.853406, 30.269578",59.853406,30.269578,89110602d53ffff
2,3,Ленинский проспект (вестибюль 1),"г.Санкт-Петербург, бульвар Новаторов, соор.1, ...","59.853514, 30.267188",59.853514,30.267188,89110602dcbffff
3,4,Ленинский проспект (вестибюль 2),"г.Санкт-Петербург, бульвар Новаторов, соор.1, ...","59.853514, 30.267188",59.853514,30.267188,89110602dcbffff
4,5,Автово,"г.Санкт-Петербург, пр. Стачек, д.90, к.2, лит.А","59.867290, 30.261277",59.867290,30.261277,89110615e4bffff
...,...,...,...,...,...,...,...
73,74,Спортивная (вестибюль 2),"г.Санкт-Петербург, 1-я линия В.О., д.58, соор....","59.947365, 30.283349",59.947365,30.283349,89110614bc3ffff
74,75,Чкаловская,"г.Санкт-Петербург, Большая Зеленина ул., д.12,...","59.960953, 30.292045",59.960953,30.292045,89110614ba3ffff
75,76,Крестовский остров,"г.Санкт-Петербург, ул. Рюхина, д.8, лит.А","59.971785, 30.259508",59.971785,30.259508,8911061499bffff
76,77,Старая деревня,"г.Санкт-Петербург, Торфяная дорога, д.4, лит.А","59.989441, 30.255142",59.989441,30.255142,891106ab203ffff


In [9]:
cian_objects['h3_9_cian'] = cian_objects.apply(lambda x:  h3.geo_to_h3(x.lat, x.lon, 9), axis=1)
cian_objects

,ID объявления,Тип,Площадь,"Площадь, м3",Метро,Адрес,Координаты,lon,lat,Этаж,Цена,"Цена, руб.",Цена/м3,Парковка,Доступ,Ссылка на объявление,Описание,h3_9_cian
0,297026179,Аренда офиса,"12.1, м2",12.1,м. Парк Победы (12 мин пешком),"Санкт-Петербург, проспект Юрия Гагарина, 19/24",30.334203 59.862199,30.334203,59.862199,-1/5,"9000 руб./ За месяц, Срок аренды - Длительный,...",9000,743.801653,NaN,NaN,https://www.cian.ru/rent/commercial/297026179,Пpедлагаютcя в аpенду офисное пoмещeние в Moск...,8911060769bffff
1,297036929,Аренда офиса,"12.2, м2",12.2,м. Балтийская (4 мин на машине),"Санкт-Петербург, Садовая улица, 107",30.287661 59.918683,30.287661,59.918683,1/3,"10000 руб./ За месяц, Срок аренды - Длительный...",10000,819.672131,NaN,пропускная система,https://www.cian.ru/rent/commercial/297036929,Предлагается в аренду офисное помещение в Адми...,8911061598fffff
2,297021255,Аренда офиса,"13.1, м2",13.1,м. Фрунзенская (11 мин пешком),"Санкт-Петербург, Измайловский проспект, 22к3",30.310101 59.910072,30.310101,59.910072,3/3,"10000 руб./ За месяц, Срок аренды - Длительный...",10000,763.358779,NaN,пропускная система,https://www.cian.ru/rent/commercial/297021255,Предлагаем в коммерческом помещение офис площа...,8911061596bffff
3,297027184,Аренда офиса,"12.2, м2",12.2,м. Ладожская (11 мин на машине),"Санкт-Петербург, 3-я Жерновская улица, 23",30.485443 59.962642,30.485443,59.962642,2/2,"12000 руб./ За месяц, Срок аренды - Длительный...",12000,983.606557,NaN,NaN,https://www.cian.ru/rent/commercial/297027184,Пpeдлагaется в аpенду офисноe помeщение в Кpaс...,89110604cabffff
4,297026569,Аренда офиса,"11.5, м2",11.5,м. Горный институт (9 мин пешком),"Санкт-Петербург, 19-я Васильевского острова ли...",30.269407 59.932482,30.269407,59.932482,-1/3,"12000 руб./ За месяц, Срок аренды - Длительный...",12000,1043.478261,NaN,NaN,https://www.cian.ru/rent/commercial/297026569,"Сдается офис площадью 11,5 кв.м.\n\nВозможно п...",89110614a13ffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7201,297496663,Аренда помещения свободного назначения,"104.0, м2",104.0,м. Комендантский проспект (9 мин на машине),"Санкт-Петербург, улица Лидии Зверевой, 9к1",30.257055 60.040636,30.257055,60.040636,1/25,"104000 руб./ За месяц, Срок аренды - Длительны...",104000,1000.000000,NaN,NaN,https://www.cian.ru/rent/commercial/297496663,Помещение находится в густонаселенном ЖК Шувал...,891106aa26fffff
7202,297235574,"Аренда торговой площади, Помещение в торговом ...","86.9, м2",86.9,м. Рыбацкое (10 мин на машине),"Санкт-Петербург, пос. Металлострой, Школьная у...",30.324429 59.793459,30.324429,59.793459,1/2,"104280 руб./ За месяц, Срок аренды - Длительны...",104280,1200.000000,NaN,NaN,https://www.cian.ru/rent/commercial/297235574,Сдается в аренду помещение на центральной улиц...,8911060002fffff
7203,290315145,Аренда офиса,"70.2, м2",70.2,м. Гостиный двор (9 мин пешком),"Санкт-Петербург, переулок Апраксин, 8",30.326091 59.928726,30.326091,59.928726,5/5,"105300 руб./ За месяц, Срок аренды - Длительны...",105300,1500.000000,Открытая,NaN,https://www.cian.ru/rent/commercial/290315145,Арт. 48364295 Аренда Коммерческого Помещения в...,891106066b7ffff
7204,297613986,Аренда помещения свободного назначения,"54.2, м2",54.2,м. Звездная (9 мин пешком),"Санкт-Петербург, улица Ленсовета, 81",30.33926 59.837797,30.339260,59.837797,1/5,"99000 руб./ За месяц, Срок аренды - Длительный...",99000,1826.568266,NaN,NaN,https://www.cian.ru/rent/commercial/297613986,Отличное помещение свободного назначения в про...,89110602b7bffff


In [10]:
yandex_objects.to_excel("yandex_objects_Sanya.xlsx", index=False)
parking_objects.to_excel("parking_objects_Sanya.xlsx", index=False)
metro_objects.to_excel("metro_objects_Sanya.xlsx", index=False)
cian_objects.to_excel("cian_objects_Sanya.xlsx", index=False)

### Население (Через OSM)

In [20]:
data_poi

,city,object,type,geometry,lat,lon,h3_9_osm
0,Санкт-Петербург,building,apartments,POINT (30.40388 60.00498),60.004981,30.403879,891106a9e6bffff
1,Санкт-Петербург,building,apartments,"POLYGON ((30.40320 60.04657, 30.40350 60.04651...",60.046459,30.403285,891106a9d77ffff
2,Санкт-Петербург,building,apartments,"POLYGON ((30.40161 60.04480, 30.40154 60.04471...",60.044930,30.400725,891106a9d73ffff
3,Санкт-Петербург,building,apartments,"POLYGON ((30.39701 60.04574, 30.39963 60.04518...",60.045510,30.398366,891106a9d73ffff
4,Санкт-Петербург,building,apartments,"POLYGON ((30.39747 60.04607, 30.39741 60.04599...",60.046401,30.397626,891106a9d0fffff
...,...,...,...,...,...,...,...
14609,Санкт-Петербург,amenity,bench,"LINESTRING (30.39300 59.94652, 30.39297 59.946...",59.946507,30.392972,89110606edbffff
14610,Санкт-Петербург,amenity,bench,"LINESTRING (30.30618 59.96646, 30.30619 59.96652)",59.966488,30.306183,891106a96cfffff
14611,Санкт-Петербург,amenity,bench,"LINESTRING (30.30637 59.96645, 30.30639 59.96651)",59.966476,30.306380,891106a96cfffff
14612,Санкт-Петербург,amenity,bench,"LINESTRING (30.47962 59.92509, 30.47964 59.925...",59.925074,30.479709,891106043bbffff


In [30]:
gdf_aparts = ox.geometries_from_place(city, {'building' : 'apartments'}).reset_index()
# полнота building:levels (этажи)
print(np.round(len(gdf_aparts['building:levels'].dropna())/len(gdf_aparts['building:levels']), 2))
# полнота building:flats (квартиры)
print(np.round(len(gdf_aparts['building:flats'].dropna())/len(gdf_aparts['building:flats']), 2))

C:\Econometrics\Temp\ipykernel_15524\1362352514.py:1: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf_aparts = ox.geometries_from_place(city, {'building' : 'apartments'}).reset_index()


0.96
0.2


In [31]:
lat_g, lon_g = get_lat_lon(gdf_aparts['geometry'])
gdf_aparts['lat'] = lat_g
gdf_aparts['lon'] = lon_g
gdf_aparts['building:levels'] = gdf_aparts['building:levels'].fillna(1)
gdf_aparts = gdf_aparts.rename(columns = {'building:levels' : 'levels'})

C:\Econometrics\Temp\ipykernel_15524\233577974.py:15: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
C:\Econometrics\Temp\ipykernel_15524\233577974.py:16: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


In [49]:
apartments = ['apartments' , 'dormitory']
houses = ['house', 'semidetached_house', 'detached', 'terrace']
people_ctn = []
for i in range(len(gdf_aparts)):
    if gdf_aparts['levels'].iloc[i] == "5; 6" or gdf_aparts['levels'].iloc[i] == "yes":
        next
    elif gdf_aparts['building'].iloc[i] in apartments:
        #print(itog_table_people['levels'].iloc[i])
        people =  int(float(gdf_aparts['levels'].iloc[i]))*10*3
        
    elif gdf_aparts['type'].iloc[i] in houses:
        
        people = int(float(gdf_aparts['levels'].iloc[i]))*3
        
    else:
        people = 'not living area'
        
    people_ctn.append(people)
    
gdf_aparts['count_people'] = people_ctn

#table_people = gdf_aparts.query("count_people != 'not living area'")
table_people['count_people'] = table_people['count_people'].astype(int)
table_people['count_people']

0        660
1        360
2        150
3        270
4        150
        ... 
14957    360
14958    360
14959    540
14960    540
14961    720
Name: count_people, Length: 14962, dtype: int32

In [53]:
table_people

,element_type,osmid,addr:flats,ref,geometry,addr:city,addr:country,addr:housenumber,addr:street,source,...,heritage,building:facade:color,min_height,note_2,postal_code,was:name,design:framework,lat,lon,count_people
0,node,9754040186,NaN,NaN,POINT (30.40388 60.00498),NaN,NaN,17,улица Верности,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.004981,30.403879,660
1,way,15806592,NaN,NaN,"POLYGON ((30.40320 60.04657, 30.40350 60.04651...",Санкт-Петербург,RU,78,Светлановский проспект,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.046459,30.403285,360
2,way,15806664,NaN,NaN,"POLYGON ((30.40161 60.04480, 30.40154 60.04471...",Санкт-Петербург,RU,14,Тимуровская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.044930,30.400725,150
3,way,15806669,NaN,NaN,"POLYGON ((30.39701 60.04574, 30.39963 60.04518...",Санкт-Петербург,RU,12,Тимуровская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.045510,30.398366,270
4,way,15806700,NaN,NaN,"POLYGON ((30.39747 60.04607, 30.39741 60.04599...",Санкт-Петербург,RU,10 к1,Тимуровская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.046401,30.397626,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14957,relation,16620394,NaN,NaN,"POLYGON ((30.50981 59.83444, 30.50977 59.83445...",Санкт-Петербург,RU,37,Шлиссельбургский проспект,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.834548,30.510136,360
14958,relation,16620396,NaN,NaN,"POLYGON ((30.50798 59.83273, 30.50794 59.83269...",Санкт-Петербург,RU,26 к1,Караваевская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.832923,30.508229,360
14959,relation,16707383,NaN,NaN,"MULTIPOLYGON (((30.33751 59.82498, 30.33750 59...",Санкт-Петербург,NaN,12 к1 с1,Среднерогатская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.824597,30.338059,540
14960,relation,16707384,NaN,NaN,"MULTIPOLYGON (((30.33762 59.82401, 30.33762 59...",Санкт-Петербург,NaN,14 к1 с1,Среднерогатская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.823631,30.338110,540


In [54]:
table_people['h3_9_population'] = table_people.apply(lambda x:  h3.geo_to_h3(x.lat, x.lon, 9), axis=1)
table_people

,element_type,osmid,addr:flats,ref,geometry,addr:city,addr:country,addr:housenumber,addr:street,source,...,building:facade:color,min_height,note_2,postal_code,was:name,design:framework,lat,lon,count_people,h3_9_population
0,node,9754040186,NaN,NaN,POINT (30.40388 60.00498),NaN,NaN,17,улица Верности,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.004981,30.403879,660,891106a9e6bffff
1,way,15806592,NaN,NaN,"POLYGON ((30.40320 60.04657, 30.40350 60.04651...",Санкт-Петербург,RU,78,Светлановский проспект,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.046459,30.403285,360,891106a9d77ffff
2,way,15806664,NaN,NaN,"POLYGON ((30.40161 60.04480, 30.40154 60.04471...",Санкт-Петербург,RU,14,Тимуровская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.044930,30.400725,150,891106a9d73ffff
3,way,15806669,NaN,NaN,"POLYGON ((30.39701 60.04574, 30.39963 60.04518...",Санкт-Петербург,RU,12,Тимуровская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.045510,30.398366,270,891106a9d73ffff
4,way,15806700,NaN,NaN,"POLYGON ((30.39747 60.04607, 30.39741 60.04599...",Санкт-Петербург,RU,10 к1,Тимуровская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.046401,30.397626,150,891106a9d0fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14957,relation,16620394,NaN,NaN,"POLYGON ((30.50981 59.83444, 30.50977 59.83445...",Санкт-Петербург,RU,37,Шлиссельбургский проспект,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.834548,30.510136,360,8911060e123ffff
14958,relation,16620396,NaN,NaN,"POLYGON ((30.50798 59.83273, 30.50794 59.83269...",Санкт-Петербург,RU,26 к1,Караваевская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.832923,30.508229,360,8911060e123ffff
14959,relation,16707383,NaN,NaN,"MULTIPOLYGON (((30.33751 59.82498, 30.33750 59...",Санкт-Петербург,NaN,12 к1 с1,Среднерогатская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.824597,30.338059,540,89110600c8bffff
14960,relation,16707384,NaN,NaN,"MULTIPOLYGON (((30.33762 59.82401, 30.33762 59...",Санкт-Петербург,NaN,14 к1 с1,Среднерогатская улица,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.823631,30.338110,540,89110600cc7ffff


In [52]:
def create_heatmap(data, lat_lon_feature):
    
    m = folium.Map(location=[sum(data['lat'])/len(data['lat']), sum(data['lon'])/len(data['lon'])], zoom_start=13, tiles='cartodbpositron')
    
    HeatMap(data[lat_lon_feature].groupby(lat_lon_feature[0:2]).sum().reset_index().values.tolist(), 
                    radius = 70, min_opacity = 0.05, max_val = int((data[lat_lon_feature[2]]).quantile([0.75])), blur=30).add_to(m)
    return m
  
# карта плотности населения
create_heatmap(table_people, ['lat', 'lon', 'count_people'])

C:\Econometrics\Temp\ipykernel_15524\3521537976.py:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  radius = 70, min_opacity = 0.05, max_val = int((data[lat_lon_feature[2]]).quantile([0.75])), blur=30).add_to(m)
C:\Econometrics\Temp\ipykernel_15524\3521537976.py:5: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(data[lat_lon_feature].groupby(lat_lon_feature[0:2]).sum().reset_index().values.tolist(),


In [55]:
table_people.to_excel("population_Sanya.xlsx", index=False)

### Население новое

In [92]:
population = pd.read_excel("population_doma.xlsx", sheet_name='Sheet1', header=0, na_values=['NA'])
population

,id,adress,coordinates,lon,lat,жил. помещения,Население,h3_9_population,h3_9_neighbors,h3_9_neighbors_1,h3_9_neighbors_2,h3_9_neighbors_3,h3_9_neighbors_4,h3_9_neighbors_5,h3_9_neighbors_6,h3_9_neighbors_7
0,9360633,"Санкт-Петербург, д. 6, стр. 1",30.314494 59.938676,30.314494,59.938676,NaN,NaN,89110606407ffff,"['8911060640fffff', '89110606417ffff', '891106...",8911060640fffff,89110606417ffff,891106064abffff,89110606433ffff,8911060643bffff,89110606403ffff,89110606407ffff
1,7033541,"Санкт-Петербург, линия. 10-я В.О., д. 5",30.277869 59.938284,30.277869,59.938284,22.0,66.0,89110614a07ffff,"['89110614a17ffff', '89110614a07ffff', '891106...",89110614a17ffff,89110614a07ffff,89110614a03ffff,89110614a33ffff,89110614a3bffff,89110614a0fffff,89110614aabffff
2,9058812,"Санкт-Петербург, линия. 10-я В.О., д. 5, стр. А",30.277869 59.938284,30.277869,59.938284,NaN,NaN,89110614a07ffff,"['89110614a17ffff', '89110614a07ffff', '891106...",89110614a17ffff,89110614a07ffff,89110614a03ffff,89110614a33ffff,89110614a3bffff,89110614a0fffff,89110614aabffff
3,7033575,"Санкт-Петербург, линия. 10-я В.О., д. 7",30.277465 59.938671,30.277465,59.938671,NaN,NaN,89110614a33ffff,"['89110614a07ffff', '89110614a23ffff', '891106...",89110614a07ffff,89110614a23ffff,89110614a37ffff,89110614a33ffff,89110614aafffff,89110614a3bffff,89110614aabffff
4,9058813,"Санкт-Петербург, линия. 10-я В.О., д. 7, стр. А",30.277465 59.938671,30.277465,59.938671,NaN,NaN,89110614a33ffff,"['89110614a07ffff', '89110614a23ffff', '891106...",89110614a07ffff,89110614a23ffff,89110614a37ffff,89110614a33ffff,89110614aafffff,89110614a3bffff,89110614aabffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22612,9548736,"Санкт-Петербург, ул. Туристская, д. 35, к. 2, ...",30.216164 60.008689,30.216164,60.008689,NaN,NaN,891106ab42bffff,"['891106ab42fffff', '891106ab42bffff', '891106...",891106ab42fffff,891106ab42bffff,891106ab477ffff,891106ab093ffff,891106ab423ffff,891106ab43bffff,891106ab467ffff
22613,9557185,"Санкт-Петербург, ул. Штурманская, д. 16, к. 1",30.303786 59.81207,30.303786,59.812070,NaN,NaN,89110600523ffff,"['8911060052bffff', '89110600523ffff', '891106...",8911060052bffff,89110600523ffff,89110600533ffff,8911060053bffff,8911060052fffff,89110600527ffff,89110600537ffff
22614,9557073,"Санкт-Петербург, п. Шушары, ул. Вишерская, д. ...",30.375507 59.802929,30.375507,59.802929,NaN,NaN,89110600e7bffff,"['89110600e73ffff', '89110601db7ffff', '891106...",89110600e73ffff,89110601db7ffff,89110600e7bffff,89110600e47ffff,89110600e63ffff,89110600e4fffff,89110600e6bffff
22615,9575018,"Санкт-Петербург, п. Шушары, ш. Колпинское, д. ...",30.474493 59.737051,30.474493,59.737051,NaN,NaN,8911060f617ffff,"['8911060f607ffff', '8911060f613ffff', '891106...",8911060f607ffff,8911060f613ffff,8911060f68fffff,8911060f617ffff,8911060f6bbffff,8911060f6abffff,8911060f603ffff


In [100]:
population['h3_9_neighbors'] = population.apply(lambda x:  list(h3.k_ring(x.h3_9_population, 1)), axis=1)
population

,id,adress,coordinates,lon,lat,жил. помещения,Население,h3_9_population,h3_9_neighbors,h3_9_neighbors_1,h3_9_neighbors_2,h3_9_neighbors_3,h3_9_neighbors_4,h3_9_neighbors_5,h3_9_neighbors_6,h3_9_neighbors_7,жил. помещения_median
0,9360633,"Санкт-Петербург, д. 6, стр. 1",30.314494 59.938676,30.314494,59.938676,NaN,NaN,89110606407ffff,"[8911060640fffff, 89110606417ffff, 891106064ab...",8911060640fffff,89110606417ffff,891106064abffff,89110606433ffff,8911060643bffff,89110606403ffff,89110606407ffff,NaN
1,7033541,"Санкт-Петербург, линия. 10-я В.О., д. 5",30.277869 59.938284,30.277869,59.938284,22.0,66.0,89110614a07ffff,"[89110614a17ffff, 89110614a07ffff, 89110614a03...",89110614a17ffff,89110614a07ffff,89110614a03ffff,89110614a33ffff,89110614a3bffff,89110614a0fffff,89110614aabffff,NaN
2,9058812,"Санкт-Петербург, линия. 10-я В.О., д. 5, стр. А",30.277869 59.938284,30.277869,59.938284,NaN,NaN,89110614a07ffff,"[89110614a17ffff, 89110614a07ffff, 89110614a03...",89110614a17ffff,89110614a07ffff,89110614a03ffff,89110614a33ffff,89110614a3bffff,89110614a0fffff,89110614aabffff,NaN
3,7033575,"Санкт-Петербург, линия. 10-я В.О., д. 7",30.277465 59.938671,30.277465,59.938671,NaN,NaN,89110614a33ffff,"[89110614a07ffff, 89110614a23ffff, 89110614a37...",89110614a07ffff,89110614a23ffff,89110614a37ffff,89110614a33ffff,89110614aafffff,89110614a3bffff,89110614aabffff,NaN
4,9058813,"Санкт-Петербург, линия. 10-я В.О., д. 7, стр. А",30.277465 59.938671,30.277465,59.938671,NaN,NaN,89110614a33ffff,"[89110614a07ffff, 89110614a23ffff, 89110614a37...",89110614a07ffff,89110614a23ffff,89110614a37ffff,89110614a33ffff,89110614aafffff,89110614a3bffff,89110614aabffff,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22612,9548736,"Санкт-Петербург, ул. Туристская, д. 35, к. 2, ...",30.216164 60.008689,30.216164,60.008689,NaN,NaN,891106ab42bffff,"[891106ab42fffff, 891106ab42bffff, 891106ab477...",891106ab42fffff,891106ab42bffff,891106ab477ffff,891106ab093ffff,891106ab423ffff,891106ab43bffff,891106ab467ffff,NaN
22613,9557185,"Санкт-Петербург, ул. Штурманская, д. 16, к. 1",30.303786 59.81207,30.303786,59.812070,NaN,NaN,89110600523ffff,"[8911060052bffff, 89110600523ffff, 89110600533...",8911060052bffff,89110600523ffff,89110600533ffff,8911060053bffff,8911060052fffff,89110600527ffff,89110600537ffff,NaN
22614,9557073,"Санкт-Петербург, п. Шушары, ул. Вишерская, д. ...",30.375507 59.802929,30.375507,59.802929,NaN,NaN,89110600e7bffff,"[89110600e73ffff, 89110601db7ffff, 89110600e7b...",89110600e73ffff,89110601db7ffff,89110600e7bffff,89110600e47ffff,89110600e63ffff,89110600e4fffff,89110600e6bffff,NaN
22615,9575018,"Санкт-Петербург, п. Шушары, ш. Колпинское, д. ...",30.474493 59.737051,30.474493,59.737051,NaN,NaN,8911060f617ffff,"[8911060f607ffff, 8911060f613ffff, 8911060f68f...",8911060f607ffff,8911060f613ffff,8911060f68fffff,8911060f617ffff,8911060f6bbffff,8911060f6abffff,8911060f603ffff,NaN


In [107]:
for index, row in population[].iterrows():
    temp = []
    if np.isnan(row['жил. помещения']) == False:
        next
    else:
        for cell in row['h3_9_neighbors']:
            if np.isnan(population.loc[(population["h3_9_population"] == cell) & (population["Население"] >= 0), ['жил. помещения']].median().iloc[0]) == True:
                next
            else:
                temp.append(population.loc[(population["h3_9_population"] == cell) & (population["Население"] >= 0), ['жил. помещения']].median().iloc[0])
        population.at[index, 'жил. помещения_median'] = round(np.mean(temp),0)
population

,id,adress,coordinates,lon,lat,жил. помещения,Население,h3_9_population,h3_9_neighbors,h3_9_neighbors_1,h3_9_neighbors_2,h3_9_neighbors_3,h3_9_neighbors_4,h3_9_neighbors_5,h3_9_neighbors_6,h3_9_neighbors_7,жил. помещения_median
0,9360633,"Санкт-Петербург, д. 6, стр. 1",30.314494 59.938676,30.314494,59.938676,NaN,NaN,89110606407ffff,"[8911060640fffff, 89110606417ffff, 891106064ab...",8911060640fffff,89110606417ffff,891106064abffff,89110606433ffff,8911060643bffff,89110606403ffff,89110606407ffff,NaN
1,7033541,"Санкт-Петербург, линия. 10-я В.О., д. 5",30.277869 59.938284,30.277869,59.938284,22.0,66.0,89110614a07ffff,"[89110614a17ffff, 89110614a07ffff, 89110614a03...",89110614a17ffff,89110614a07ffff,89110614a03ffff,89110614a33ffff,89110614a3bffff,89110614a0fffff,89110614aabffff,20.0
2,9058812,"Санкт-Петербург, линия. 10-я В.О., д. 5, стр. А",30.277869 59.938284,30.277869,59.938284,NaN,NaN,89110614a07ffff,"[89110614a17ffff, 89110614a07ffff, 89110614a03...",89110614a17ffff,89110614a07ffff,89110614a03ffff,89110614a33ffff,89110614a3bffff,89110614a0fffff,89110614aabffff,NaN
3,7033575,"Санкт-Петербург, линия. 10-я В.О., д. 7",30.277465 59.938671,30.277465,59.938671,NaN,NaN,89110614a33ffff,"[89110614a07ffff, 89110614a23ffff, 89110614a37...",89110614a07ffff,89110614a23ffff,89110614a37ffff,89110614a33ffff,89110614aafffff,89110614a3bffff,89110614aabffff,NaN
4,9058813,"Санкт-Петербург, линия. 10-я В.О., д. 7, стр. А",30.277465 59.938671,30.277465,59.938671,NaN,NaN,89110614a33ffff,"[89110614a07ffff, 89110614a23ffff, 89110614a37...",89110614a07ffff,89110614a23ffff,89110614a37ffff,89110614a33ffff,89110614aafffff,89110614a3bffff,89110614aabffff,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22612,9548736,"Санкт-Петербург, ул. Туристская, д. 35, к. 2, ...",30.216164 60.008689,30.216164,60.008689,NaN,NaN,891106ab42bffff,"[891106ab42fffff, 891106ab42bffff, 891106ab477...",891106ab42fffff,891106ab42bffff,891106ab477ffff,891106ab093ffff,891106ab423ffff,891106ab43bffff,891106ab467ffff,NaN
22613,9557185,"Санкт-Петербург, ул. Штурманская, д. 16, к. 1",30.303786 59.81207,30.303786,59.812070,NaN,NaN,89110600523ffff,"[8911060052bffff, 89110600523ffff, 89110600533...",8911060052bffff,89110600523ffff,89110600533ffff,8911060053bffff,8911060052fffff,89110600527ffff,89110600537ffff,NaN
22614,9557073,"Санкт-Петербург, п. Шушары, ул. Вишерская, д. ...",30.375507 59.802929,30.375507,59.802929,NaN,NaN,89110600e7bffff,"[89110600e73ffff, 89110601db7ffff, 89110600e7b...",89110600e73ffff,89110601db7ffff,89110600e7bffff,89110600e47ffff,89110600e63ffff,89110600e4fffff,89110600e6bffff,NaN
22615,9575018,"Санкт-Петербург, п. Шушары, ш. Колпинское, д. ...",30.474493 59.737051,30.474493,59.737051,NaN,NaN,8911060f617ffff,"[8911060f607ffff, 8911060f613ffff, 8911060f68f...",8911060f607ffff,8911060f613ffff,8911060f68fffff,8911060f617ffff,8911060f6bbffff,8911060f6abffff,8911060f603ffff,NaN


In [117]:
def create_heatmap(data, lat_lon_feature):
    
    m = folium.Map(location=[sum(data['lat'])/len(data['lat']), sum(data['lon'])/len(data['lon'])], zoom_start=13, tiles='cartodbpositron')
    
    HeatMap(data[lat_lon_feature].groupby(lat_lon_feature[0:2]).sum().reset_index().values.tolist(), 
                    radius = 70, min_opacity = 0.05, max_val = int((data[lat_lon_feature[2]]).quantile([0.75])), blur=30).add_to(m)
    return m
  
# карта плотности населения
create_heatmap(population, ['lat', 'lon', 'population'])

C:\Econometrics\Temp\ipykernel_20432\3774682046.py:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  radius = 70, min_opacity = 0.05, max_val = int((data[lat_lon_feature[2]]).quantile([0.75])), blur=30).add_to(m)
C:\Econometrics\Temp\ipykernel_20432\3774682046.py:5: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(data[lat_lon_feature].groupby(lat_lon_feature[0:2]).sum().reset_index().values.tolist(),


In [110]:
population.to_excel("population_doma_full_1.xlsx", index=False)